<a href="https://colab.research.google.com/github/cat0ros/robotics-control-3DOF/blob/master/fk_ik_python_3dof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по управлению роботами №1
Выполнил: Романов Ростислав, группа: АДБ-20-09

<img src= "https://drive.google.com/uc?export=view&id=1PpzfCIzka8X_ljyS4piV1AKeBfmu2AZ3" alt="inverse" width="400" />

***Рисунок 1: Пространственное представление схемы манипулятора***

<img src= "https://drive.google.com/uc?export=view&id=1zDL-aUGP3qxmC3ibMRZ7z9wZ6hDe7knu" alt="inverse" width="400" />

***Рисунок 2: Кинематическая схема манипулятора***

## Параметры и описание робота
Длины звеньев робота:

In [407]:
links_length = [1.5, 1.3, 2.2]

## Библиотеки для работы

In [408]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from numpy.linalg import inv
import math
import sympy as sp
from IPython.display import HTML, Math, Latex
%matplotlib notebook

## Решение прямой задачи кинематики
**Составление DH-матриц**

Матрица Денавита-Хартенберга:

<img src= "https://wikimedia.org/api/rest_v1/media/math/render/svg/6963d0c47a3a894ff0719c8df348d188b996074e" alt="inverse" width="400" />

Функции вычисления матриц преобразований:

In [409]:
def rz(a):
    return sp.Matrix([
        [sp.cos(a), -sp.sin(a), 0, 0],
        [sp.sin(a), sp.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def ry(a):
    return sp.Matrix([
        [sp.cos(a), 0, sp.sin(a), 0],
        [0, 1, 0, 0],
        [-sp.sin(a), 0, sp.cos(a), 0],
        [0, 0, 0, 1]
    ])

def rx(a):
    return sp.Matrix([
        [1, 0, 0, 0],
        [0, sp.cos(a), -sp.sin(a), 0],
        [0, sp.sin(a), sp.cos(a), 0],
        [0, 0, 0, 1]
    ])

def trs(x, y, z):
    return sp.Matrix([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

def vec(x, y, z):
    return sp.Matrix([
        [x],
        [y],
        [z],
        [1]
    ])

def dh(theta, d, alpha, a):
    return rz(theta) * trs(0, 0, d) * rx(alpha) * trs(a, 0, 0)


Реализуем функцию, которая будет возвращать матрицу под соответствующие DH-параметры:

In [410]:
def get_dh_matrix(theta, alpha, r, d):
  alpha = (np.pi/180) * alpha
  theta = (np.pi/180) * theta

  ct = np.cos(theta)
  st = np.sin(theta)
  sa = np.sin(alpha)
  ca = np.cos(alpha)

  dh_matrix = np.array([
      [ct,  -st * ca, st * sa, r * ct],
      [st, ct *  ca, -ct * sa, r * st],
      [0, sa, ca, d],
      [0, 0, 0, 1]
  ])

  return dh_matrix

Расставим системы координат и составим таблицу, а по ней итоговую матрицу преобразований:

<img src= "https://drive.google.com/uc?export=view&id=18EiOZVcCKHVvgkmfPqdece4xYVxcEvYH" alt="inverse" width="600" />

Заметим, что нулевое положение робота будет слева, тогда для изображения справа повернем вокруг оси Z концевую точку на 90 градусов в положительном направлении.

In [411]:
def forward_kinematics(generilized):
  r = [0, 0, links_length[2]]
  d = [links_length[0], links_length[1] + generilized[1], 0]
  theta = [generilized[0] - 90, 90, generilized[2] + 90]
  alpha = [-90, 90, 0]

  #links_length[2]

  matrix = np.eye(4)
  for i in range(len(theta)):
    matrix = matrix @ get_dh_matrix(theta[i], alpha[i], r[i], d[i])

  return matrix

theta1, d1, alpha1, a1 = sp.symbols("theta_1, l_1, alpha_1, a_1")
theta2, d2, alpha2, a2 = sp.symbols("theta_2, d_2, alpha_2, a_2")
theta3, d3, alpha3, a3 = sp.symbols("theta_3, d_3, alpha_3, l_3")
dh1 = dh(theta1 - sp.pi / 2, d1, -sp.pi / 2, 0)
dh2 = dh(sp.pi / 2, d2 + sp.symbols("l_2"), sp.pi / 2, 0)
dh3 = dh(theta3 + sp.pi / 2, 0, 0, a3)

dh_op = dh1 * dh2 * dh3

Результирующая матрица для данной кинематической схемы:

In [412]:
dh_op

Matrix([
[cos(theta_1)*cos(theta_3), -sin(theta_3)*cos(theta_1),  sin(theta_1), l_3*cos(theta_1)*cos(theta_3) + (d_2 + l_2)*cos(theta_1)],
[sin(theta_1)*cos(theta_3), -sin(theta_1)*sin(theta_3), -cos(theta_1), l_3*sin(theta_1)*cos(theta_3) + (d_2 + l_2)*sin(theta_1)],
[             sin(theta_3),               cos(theta_3),             0,                                   l_1 + l_3*sin(theta_3)],
[                        0,                          0,             0,                                                        1]])

Первая матрица преобразований:

In [413]:
sp.simplify(dh1)

Matrix([
[ sin(theta_1),  0, cos(theta_1),   0],
[-cos(theta_1),  0, sin(theta_1),   0],
[            0, -1,            0, l_1],
[            0,  0,            0,   1]])

Вторая матрица преобразований:

In [414]:
sp.simplify(dh2)

Matrix([
[0, 0, 1,         0],
[1, 0, 0,         0],
[0, 1, 0, d_2 + l_2],
[0, 0, 0,         1]])

Третья матрица преобразований:

In [415]:
sp.simplify(dh3)

Matrix([
[-sin(theta_3), -cos(theta_3), 0, -l_3*sin(theta_3)],
[ cos(theta_3), -sin(theta_3), 0,  l_3*cos(theta_3)],
[            0,             0, 1,                 0],
[            0,             0, 0,                 1]])

Решим ПЗК для вектора обобщенных координат q = [30, 2.2, 90]

In [416]:
forward_kinematics([30, 2.2, 90])

array([[ 2.24125920e-17, -8.66025404e-01,  5.00000000e-01,
         3.03108891e+00],
       [ 8.36449319e-17, -5.00000000e-01, -8.66025404e-01,
         1.75000000e+00],
       [ 1.00000000e+00,  6.12323400e-17,  6.12323400e-17,
         3.70000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

Решение в матричном виде:

In [418]:
dh1_1 = dh(sp.pi / 6 - sp.pi / 2, links_length[0], -sp.pi / 2, 0)
dh2_2 = dh(sp.pi / 2, 2.2 + links_length[1], sp.pi / 2, 0)
dh3_3 = dh(sp.pi / 2 + sp.pi / 2, 0, 0, links_length[2])

dh_op_ = dh1_1 * dh2_2 * dh3_3

dh_op_

Matrix([
[0, -sqrt(3)/2,        1/2, 1.75*sqrt(3)],
[0,       -1/2, -sqrt(3)/2,         1.75],
[1,          0,          0,          3.7],
[0,          0,          0,            1]])

# Решение обратной задачи кинематики

Рассмотрид вид сверху исходной кинематической схемы:

<img src= "https://drive.google.com/uc?export=view&id=16u0Gn5cK_Mwyt6NhC_CyctTbbU0kwyRr" alt="inverse" width="200" />

Заметим следующую зависимость:

In [335]:
display(Math(r'\Theta_{1} = \text{arctan2}(y_0, x_0)'))

<IPython.core.display.Math object>

Не является возможным найти сразу решение для второй обобщенной координаты, так как нам неизвестна проекция длины третьего звена.

Найдем ее по виду сбоку нашего манипулятора и найдем высоту обозначенную как b:

<img src= "https://drive.google.com/uc?export=view&id=1oQSe0p5Er6_KpYB8kVftatvfdA_2yEO7" alt="inverse" width="400" />

In [336]:
display(Math(r'b = z - l_{1}'))

<IPython.core.display.Math object>

Тогда становится известна проекция длины третьего звена на плоскость X0Y0:

In [337]:
display(Math(r'l_{3_{x_{0}y_{0}}} = l_{3}*cos(\Theta_{3})'))

<IPython.core.display.Math object>

Найдем этот угол:

In [338]:
display(Math(r'\Theta_{3} = arcsin(\frac{z - l1}{l3})'))
display(Math(r'\Theta_{3} = \pi - arcsin(\frac{z - l1}{l3})'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

В результате мы получили два решения.

Так как от найденного угла зависит проекция третьего звена на плоскость X0Y0, вычислим вторую обобщенную координату:

In [339]:
display(Math(r'd2 = \sqrt{x_{0}^{2} + y_{0}^{2}} - l_{2} - l_{3} * cos(\Theta_{3})'))

<IPython.core.display.Math object>

Аналитические выражения для решения ОЗК получены.

In [340]:
def IK(x, y, z, l1, l2, l3):
  theta1 = np.arctan2(y, x)
  theta3_1 = np.arcsin((z - l1) / l3)
  theta3_2 = np.pi - np.arcsin((z - l1) / l3)
  theta3_sol = [theta3_1, theta3_2]
  sqrt_xy = np.float64((x**2 + y**2)**(1/2))
  l3_x0y0_1 = np.float64(l3) * np.cos(theta3_1)
  l3_x0y0_2 = np.float64(l3) * np.cos(theta3_2)
  d2_1 = sqrt_xy - l3_x0y0_1 - l2
  d2_2 = sqrt_xy - l3_x0y0_2 - l2
  d2_sol = [d2_1, d2_2]

  for i in range(len(theta3_sol)):
    if (np.round(d2_sol[i], 5) >= 0):
      q = [np.round(np.rad2deg(theta1), 2), np.round(d2_sol[i], 5), np.round(np.rad2deg(theta3_sol[i]), 2)]
      yield q

Решим прямую задачу кинематики для обобщенных координат: [30, 2.2, 90]

In [341]:
fk_sol = forward_kinematics([30, 2.2, 90])
fk_sol

array([[ 2.24125920e-17, -8.66025404e-01,  5.00000000e-01,
         3.03108891e+00],
       [ 8.36449319e-17, -5.00000000e-01, -8.66025404e-01,
         1.75000000e+00],
       [ 1.00000000e+00,  6.12323400e-17,  6.12323400e-17,
         3.70000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

Вычислим позицию концевой точки, домножив на вектор-столбец: [0,0,0,1]

In [342]:
tcp_pos = fk_sol @ np.array([[0], [0], [0], [1]])
tcp_pos

array([[3.03108891],
       [1.75      ],
       [3.7       ],
       [1.        ]])

In [343]:
x_f, y_f, z_f = tcp_pos[0, 0], tcp_pos[1, 0], tcp_pos[2, 0]
l1, l2, l3 = links_length[0], links_length[1], links_length[2]

In [344]:
x_f

3.031088913245535

In [345]:
y_f

1.7500000000000007

In [346]:
z_f

3.7

Переберем все решения ОЗК для данной точки и запишем их. Решения ОЗК должны совпать с тем, что мы задавали вначале в функции ПЗК в качестве обобщенных координат. Задавали вектор: [30, 2.2, 90]

In [347]:
for i in IK(x_f, y_f, z_f, l1, l2, l3):
  print(i)

[30.0, 2.2, 90.0]
[30.0, 2.2, 90.0]


Пометка: для угла 90 градусов решение ОЗК в данном случае однозначно.

Проверка выполнена.

Подставим позицию конкретной точки и вычислим для нее ОЗК (в соответствии с вариантом): [0.4,0.3, 0.1]

In [348]:
x_v = 0.4
y_v = 0.3
z_v = 0.1

for i in IK(x_v, y_v, z_v, l1, l2, l3):
  print(i)

[36.87, 0.89706, 219.52]


# Прямая задача по скорости